In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Hit:13 http://ppa.launchpad.net/

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-19 12:39:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-07-19 12:39:31 (6.54 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-D2-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
## Load Data

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

**Deliverable 2**

In [ ]:
# Filter the data and create a new DataFrame or table to retrieve all the rows
# where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful 
# and to avoid having division by zero errors later on.(used 10 to try to get appropriate data)
totalvotes_df = df.filter("total_votes>=10")
totalvotes_df.count()

223274

In [ ]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table 
# to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%(used 30% to see whether i could get some sections to run)
helpful_50_df = totalvotes_df.filter("helpful_votes/total_votes>=.30")
helpful_50_df.count()

194191

In [ ]:
print(helpful_50_df.count())

194191


In [ ]:
vine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "Y")
vine_review_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [ ]:
novine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "N")
novine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    4514126|R39ESX43X1SA5T|B006NG0GXQ|     402593554|Forever His: A Ti...|Digital_Ebook_Pur...|          1|           36|         47|   N|                Y|   Completely absurd|From the time tha...| 2013-09-09|
|         US|   45292583|R31QRZ3JAMPBB4|B0078T6YHY|     267532424|A Lady in Defianc...|Digital_Ebook_Pur

In [ ]:
from pyspark.sql.functions import count
# Total number of reviews for each dataframe from Step 3 and Step 4:
vine_review_count = vine_review_df.count()
print("Total Number of Reviews PAID and 'helpful': %f" % vine_review_count)

novine_review_count = novine_review_df.count()
print("Total Number of Reviews NON-PAID and 'helpful': %f" % novine_review_count)

Total Number of Reviews PAID and 'helpful': 0.000000
Total Number of Reviews NON-PAID and 'helpful': 194191.000000


In [ ]:
# Number of 5-star Reviews for above Paid and Non-Paid dataframes
star5_vine_df = vine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews PAID and 'helpful': %f" % star5_vine_df.count())


star5_novine_df = novine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews NON-PAID and 'helpful': %f" % star5_novine_df.count())

Total Number of 5-star Reviews PAID and 'helpful': 0.000000
Total Number of 5-star Reviews NON-PAID and 'helpful': 24673.000000


In [ ]:
# Percentage of 5-star Reviews for above Paid and Non-Paid dataframes
#star5_vine_pct = (star5_vine_df.count()/vine_review_count)
#print("Percent of 5-Star Digital_Ebook_Purchase reviews from PAID, 'helpful' dataset: %f" % star5_vine_pct)

star5_novine_pct = (star5_novine_df.count()/novine_review_count)
print("Percent of 5-Star Digital_Ebook_Purchase reviews from NON-PAID, 'helpful' dataset: %f" % star5_novine_pct)

Percent of 5-Star Digital_Ebook_Purchase reviews from NON-PAID, 'helpful' dataset: 0.127055


In [ ]:
# Total number of ALL reviews
total_reviews_count = df.count()
print("Total  Digital_Ebook_Purchase reviews: %f" % total_reviews_count)

help_reviews_count = helpful_50_df.count()
print("Total Number of 'Helpful' Digital_Ebook_Purchase reviews: %f" % help_reviews_count)

Total  Digital_Ebook_Purchase reviews: 5101693.000000
Total Number of 'Helpful' Digital_Ebook_Purchase reviews: 194191.000000


In [ ]:
#Total number of ALL 5-star reviews
star5_df = df.filter(df["star_rating"] == '5')
star5_df.show()

star5_count = star5_df.count()
print("Total Number of 5-Star Digital_Ebook_Purchase reviews: %f" % star5_count)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   39601147| R7DRFHC0F71O0|B00EA3L35O|     535606445|Mary had A Sleepy...|Digital_Ebook_Pur...|          5|            0|          0|   N|                N|This Sleepy Sheep...|I had the opportu...| 2013-09-09|
|         US|   17351407|R27LUKEXU3KBXQ|B00BL3JV50|     240053004|          Starstruck|Digital_Ebook_Pur

In [ ]:
#Total number of 5-star HELPFUL reviews
star5_help_df = helpful_50_df.filter(helpful_50_df["star_rating"] == '5')
star5_help_df.show()

star5_help_count = star5_help_df.count()
print("Total Number of 5-Star 'Helpful' Digital_Ebook_Purchase reviews: %f" % star5_help_count)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12175486|R23FKBEXURC3SN|B00F225YK2|     953280558|Very Bad Things (...|Digital_Ebook_Pur...|          5|           28|         36|   N|                N|Holy Hotness, Bla...|Another Fave of 2...| 2013-09-09|
|         US|   51118055|R2HYOTFLI8HLZF|B006B7LSDY|     130327682|The Non-Runner's ...|Digital_Ebook_Pur

In [ ]:
# Percentage of ALL 5-star reviews paid vs non-paid 
# 5-star Paid reviews 
vine_star5_all_df = star5_df.filter(star5_df["vine"] == "Y")
vine_star5_all_count = vine_star5_all_df.count()

vine_star5_all_pct = (vine_star5_all_count/star5_count)
print("Number of 5-Star Digital_Ebook_Purchase reviews PAID: %f" % vine_star5_all_count)
print("Percent of 5-Star Digital_Ebook_Purchase reviews PAID: %f" % vine_star5_all_pct)
 
#5-star Non-Paid reviews
novine_star5_all_df = star5_df.filter(star5_df["vine"] == "N")
novine_star5_all_count = novine_star5_all_df.count()

novine_star5_all_pct = novine_star5_all_count/star5_count
print("Number of Digital_Ebook_Purchase reviews NON-PAID: %f" % novine_star5_all_count)
print("Percent of 5-Star Digital_Ebook_Purchase reviews NON-PAID: %f" % novine_star5_all_pct)

Number of 5-Star Digital_Ebook_Purchase reviews PAID: 9.000000
Percent of 5-Star Digital_Ebook_Purchase reviews PAID: 0.000003
Number of Digital_Ebook_Purchase reviews NON-PAID: 2952604.000000
Percent of 5-Star Digital_Ebook_Purchase reviews NON-PAID: 0.999997


In [ ]:
# Percentage of HELPFUL 5-star reviews paid vs non-paid (% of 5 star reviews in Step 3 dataframe) 
# 5-star Paid "helpful" reviews 
vine_star5_help_df = star5_help_df.filter(star5_help_df["vine"] == "Y")
vine_star5_help_count = vine_star5_help_df.count()

vine_star5_help_pct = (vine_star5_help_count/star5_help_count)
print("Number of 5-Star, 'Helpful' Digital_Ebook_Purchase reviews PAID: %f" % vine_star5_help_count)
print("Percent of 5-Star,'Helpful'Digital_Ebook_Purchase reviews PAID: %f" % vine_star5_help_pct)
 
#5-star Non-Paid "helpful" reviews
novine_star5_help_df = star5_help_df.filter(star5_help_df["vine"] == "N")
novine_star5_help_count = novine_star5_help_df.count()

novine_star5_help_pct = novine_star5_help_count/star5_help_count
print("Number of 5-Star, 'Helpful' Digital_Ebook_Purchase reviews NON-PAID: %f" % novine_star5_help_count)
print("Percent of 5-Star,'Helpful'Digital_Ebook_Purchase reviews NON-PAID: %f" % novine_star5_help_pct)

Number of 5-Star, 'Helpful' Digital_Ebook_Purchase reviews PAID: 0.000000
Percent of 5-Star,'Helpful'Digital_Ebook_Purchase reviews PAID: 0.000000
Number of 5-Star, 'Helpful' Digital_Ebook_Purchase reviews NON-PAID: 75488.000000
Percent of 5-Star,'Helpful'Digital_Ebook_Purchase reviews NON-PAID: 1.000000
